# SPARQL Query

In [ ]:
sparql = SPARQLWrapper("https://publications.europa.eu/webapi/rdf/sparql")
sparql.setReturnFormat(JSON)

In [ ]:
query = """
PREFIX cdm: <http://publications.europa.eu/ontology/cdm#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT 
  DISTINCT (GROUP_CONCAT(DISTINCT STR(?work); SEPARATOR=",") AS ?cellarURIs)
  (GROUP_CONCAT(DISTINCT STR(?title_); SEPARATOR=",") AS ?title)
  ?langIdentifier
  (GROUP_CONCAT(DISTINCT STR(?mtype); SEPARATOR=",") AS ?mtypes)
  (GROUP_CONCAT(DISTINCT STR(?thumbnail); SEPARATOR=",") AS ?thumbnails)
  (GROUP_CONCAT(DISTINCT STR(?resType); SEPARATOR=",") AS ?workTypes)
  (GROUP_CONCAT(DISTINCT STR(?agentName); SEPARATOR=",") AS ?authors)
  (GROUP_CONCAT(DISTINCT STR(?privateAgentName); SEPARATOR=";") AS ?privateAuthors)
  ?date
  (GROUP_CONCAT(DISTINCT STR(?subjectLabel); SEPARATOR=",") AS ?subjects)
  (GROUP_CONCAT(DISTINCT STR(?workId_); SEPARATOR=",") AS ?workIds)
WHERE {
  GRAPH ?gw {
    ?work rdf:type ?resType ;
          cdm:work_date_document ?date ;
          cdm:work_id_document ?workId_ ;
          cdm:work_is_about_concept_eurovoc ?subject .
    FILTER(?resType = cdm:resource_legal)
    FILTER(xsd:date(?date) < "2023-01-01"^^xsd:date)
    
    GRAPH ?gs {
      ?subject skos:prefLabel ?subjectLabel .
      FILTER(LANG(?subjectLabel) = "en")
    }
  }

  GRAPH ?eg {
    ?exp cdm:expression_belongs_to_work ?work ;
         cdm:expression_title ?title_ ;
         cdm:expression_uses_language ?lg .
    FILTER(LANG(?title_) = "en" || LANG(?title_) = "eng" || LANG(?title_) = "")

    GRAPH ?lgc {
      ?lg dc:identifier ?langIdentifier .
      FILTER(STR(?langIdentifier) = "ENG")
    }
  }

  GRAPH ?gm {
    ?manif cdm:manifestation_manifests_expression ?exp ;
           cdm:manifestation_type ?mtype .
    OPTIONAL { ?manif cdm:manifestation_has_thumbnail ?thumbnail }
  }

  OPTIONAL {
    GRAPH ?gagent {
      {
        ?work cdm:work_contributed_to_by_agent ?agent .
      } UNION {
        ?work cdm:work_created_by_agent ?agent .
      } UNION {
        ?work cdm:work_authored_by_agent ?agent .
      }
    }
    GRAPH ?ga {
      ?agent skos:prefLabel ?agentName .
      FILTER(LANG(?agentName) = "en")
    }
  }

  OPTIONAL {
    GRAPH ?persAuthor {
      {
        ?work cdm:work_contributed_to_by_agent ?privateAgent .
      } UNION {
        ?work cdm:work_authored_by_agent ?privateAgent .
      }
      ?privateAgent rdf:type cdm:person ;
                    cdm:agent_name ?privateAgentName .
    }
  }
}
GROUP BY ?work ?date ?langIdentifier
ORDER BY DESC(?date)
LIMIT 100
"""

In [ ]:
query = """
prefix cdm: <http://publications.europa.eu/ontology/cdm#>
prefix purl: <http://purl.org/dc/elements/1.1/>
select distinct ?item ?date ?class
where {
    ?work cdm:date_creation_legacy ?date.
    ?w a ?class .
    ?expr cdm:expression_belongs_to_work ?work ;
        cdm:expression_uses_language ?lang .
    ?lang purl:identifier ?langCode .
    ?manif cdm:manifestation_manifests_expression ?expr;
        cdm:manifestation_type "pdfa1a".
    ?item cdm:item_belongs_to_manifestation ?manif.
    FILTER (
        ?date > "2016-05-23T10:20:13+05:30"^^xsd:dateTime
        AND
        ?date < "2020-05-23T10:20:13+05:30"^^xsd:dateTime
    ).
    FILTER(
        ?class in (
        <http://publications.europa.eu/ontology/cdm#document_cjeu>,
        <http://publications.europa.eu/ontology/cdm#case-law>,
        <http://publications.europa.eu/ontology/cdm#summary_caselaw>,
        <http://publications.europa.eu/ontology/cdm#summary_caselaw_jure>
        )
    )
    FILTER(STR(?langCode) = "ENG")
}
LIMIT 10
"""

In [ ]:
### Select format? = html or xhtml


query = """
prefix cdm: <http://publications.europa.eu/ontology/cdm#>
prefix purl: <http://purl.org/dc/elements/1.1/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

select distinct ?item ?expr_title ?celex ?date ?class ?className ?authorName #  ?subject
where {
    ?work cdm:work_date_document ?date ;
        rdf:type cdm:resource_legal .
    ?w a ?class .
    ?class rdfs:subClassOf cdm:resource_legal .
    ?expr cdm:expression_belongs_to_work ?work ;
        cdm:expression_title ?expr_title ;
        cdm:expression_uses_language ?lang .
    ?lang purl:identifier ?langCode .
    ?manif cdm:manifestation_manifests_expression ?expr;
        # cdm:manifestation_type "pdfa1a".
        cdm:manifestation_type "xhtml".
    ?item cdm:item_belongs_to_manifestation ?manif.
    
    FILTER (
        ?date > "2000-01-01T23:59:59+08:00"^^xsd:dateTime
        AND
        ?date < "2025-08-01T23:59:59+08:00"^^xsd:dateTime
    ).

    FILTER(STR(?langCode) = "ENG")

    OPTIONAL { ?work cdm:work_id_celex ?celex . }
    
    BIND(
        IF(CONTAINS(STR(?class), "#"),
            STRAFTER(STR(?class), "#"),
            STRAFTER(STR(?class), "/")
        ) AS ?className
    )
    
    OPTIONAL {
        GRAPH ?gagent {
            {
            ?work cdm:work_contributed_to_by_agent ?author .
            } UNION {
            ?work cdm:work_created_by_agent ?author .
            } UNION {
            ?work cdm:work_authored_by_agent ?author .
            }
        }
    
        
    OPTIONAL {
        GRAPH ?glabel {
        ?author skos:prefLabel ?authorName .
        FILTER(LANG(?authorName) = "en")
        }
    }
    }

}
ORDER BY ?date
"""

In [ ]:
from rdflib import Namespace
Namespace("http://publications.europa.eu/ontology/cdm#")

Namespace('http://publications.europa.eu/ontology/cdm#')

In [ ]:
sparql.setQuery(query)
results = sparql.query().convert()
results["results"]["bindings"]

[{'item': {'type': 'uri',
   'value': 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1'},
  'expr_title': {'type': 'literal',
   'value': 'Opinion of Advocate General Sharpston delivered on 13 March 2008. # Ecotrade SpA v Agenzia delle Entrate - Ufficio di Genova 3. # Reference for a preliminary ruling: Commissione tributaria provinciale di Genova - Italy. # Sixth VAT Directive - Reverse charge procedure - Right to deduct - Time-bar - Irregularity in accounts and tax returns affecting transactions subject to the reverse charge procedure. # Joined cases C-95/07 and C-96/07.'},
  'date': {'type': 'typed-literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#date',
   'value': '2008-03-13'},
  'class': {'type': 'uri',
   'value': 'http://publications.europa.eu/ontology/cdm#act_consolidated'},
  'className': {'type': 'literal', 'value': 'act_consolidated'},
  'authorName': {'type': 'literal',
   'xml:lang': 'en',
   'value': 'Court of Ju

In [ ]:
for result in results["results"]["bindings"]:
    print(result["celex"]["value"])

KeyError: 'celex'

In [ ]:
def clean_results(results):
    results_clean = [
        {k:v["value"] for k,v in item.items()}
        for item in results["results"]["bindings"]
    ]
    results_clean = pd.DataFrame(results_clean)
    return results_clean

In [ ]:
items = clean_results(results)
print(items.shape[0])
items.head(3)

32


,item,expr_title,date,class,className,authorName
0,http://publications.europa.eu/resource/cellar/...,Opinion of Advocate General Sharpston delivere...,2008-03-13,http://publications.europa.eu/ontology/cdm#act...,act_consolidated,Court of Justice
1,http://publications.europa.eu/resource/cellar/...,Opinion of Advocate General Sharpston delivere...,2008-03-13,http://publications.europa.eu/ontology/cdm#act...,act_preparatory,Court of Justice
2,http://publications.europa.eu/resource/cellar/...,Opinion of Advocate General Sharpston delivere...,2008-03-13,http://publications.europa.eu/ontology/cdm#agr...,agreement_international,Court of Justice


In [ ]:
items['item'].to_list()

['http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1',
 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.0

In [ ]:
results["results"]["bindings"]

[{'item': {'type': 'uri',
   'value': 'http://publications.europa.eu/resource/cellar/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02/DOC_1'},
  'expr_title': {'type': 'literal',
   'value': 'Opinion of Advocate General Sharpston delivered on 13 March 2008. # Ecotrade SpA v Agenzia delle Entrate - Ufficio di Genova 3. # Reference for a preliminary ruling: Commissione tributaria provinciale di Genova - Italy. # Sixth VAT Directive - Reverse charge procedure - Right to deduct - Time-bar - Irregularity in accounts and tax returns affecting transactions subject to the reverse charge procedure. # Joined cases C-95/07 and C-96/07.'},
  'date': {'type': 'typed-literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#date',
   'value': '2008-03-13'},
  'class': {'type': 'uri',
   'value': 'http://publications.europa.eu/ontology/cdm#act_consolidated'},
  'className': {'type': 'literal', 'value': 'act_consolidated'},
  'authorName': {'type': 'literal',
   'xml:lang': 'en',
   'value': 'Court of Ju

In [ ]:
os.makedirs("data/api_download", exist_ok=True)
# Call to the  API to download the digital file
unique_items = set([r["item"]["value"] for r in results["results"]["bindings"]])
for doc_url in unique_items:
    item = '.'.join(doc_url.split("/")[-2:])
    file_name = "data/api_download/" + item + ".xhtml"
    try:
        response = requests.get(doc_url)
    except Exception as e:
        print(f"Error downloading {doc_url}: {e}")
        continue
    
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(response.text)
        print(f"Downloaded: {file_name}")

Downloaded: data/api_download/ea3bdcba-e537-4749-b2dc-4457734dbfcf.0002.02.DOC_1.xhtml
Downloaded: data/api_download/3d4572e0-3dcd-432d-85ad-ad918607176b.0020.02.DOC_1.xhtml


# RSS Feed

In [ ]:
import os
import requests
import feedparser
import json
import pandas as pd
from dotenv import load_dotenv
from pandas import json_normalize
from datetime import datetime, timezone
load_dotenv(override=True)

True

In [24]:
pd.set_option('display.max_rows', 50)

In [2]:
# Replace this with your EUR-Lex RSS feed URL
rss_url = "https://eur-lex.europa.eu/EN/display-feed.rss?myRssId=zqe48ppy80IwdPmk3HJVPsD4fM281XNaoMTLQ6ifL58%3D"

# Parse the RSS feed
feed = feedparser.parse(rss_url)

# Loop through the latest entries
for entry in feed.entries:
    title = entry.title
    link = entry.link
    published = entry.published  # e.g., "Fri, 29 Aug 2025 12:00:00 GMT"

    print(f"Title: {title}")
    print(f"Link: {link}")
    print(f"Published: {published}")
    print("-" * 50)

Title: CELEX:32025D1734: Decision (EU) 2025/1734 of the European Central Bank of 31 July 2025 on safeguards in relation to access by central counterparties to Eurosystem overnight credit in TARGET (ECB/2025/29)
Link: https://eur-lex.europa.eu/./legal-content/AUTO/?uri=CELEX:32025D1734
Published: Wed, 13 Aug 2025 00:00:00 +0200
--------------------------------------------------
Title: CELEX:32025R0791: Commission Delegated Regulation (EU) 2025/791 of 23 April 2025 supplementing Directive 2013/36/EU of the European Parliament and of the Council with regard to regulatory technical standards specifying the general conditions for the functioning of supervisory colleges, and repealing Commission Delegated Regulation (EU) 2016/98
Link: https://eur-lex.europa.eu/./legal-content/AUTO/?uri=CELEX:32025R0791
Published: Fri, 08 Aug 2025 00:00:00 +0200
--------------------------------------------------
Title: CELEX:32025R0790: Commission Implementing Regulation (EU) 2025/790 of 23 April 2025 laying 

In [3]:
feed.entries[0]

{'title': 'CELEX:32025D1734: Decision (EU) 2025/1734 of the European Central Bank of 31\xa0July 2025 on safeguards in relation to access by central counterparties to Eurosystem overnight credit in TARGET (ECB/2025/29)',
 'title_detail': {'type': 'text/plain',
  'language': 'en',
  'base': 'https://eur-lex.europa.eu/EN/display-feed.rss?myRssId=zqe48ppy80IwdPmk3HJVPsD4fM281XNaoMTLQ6ifL58%3D',
  'value': 'CELEX:32025D1734: Decision (EU) 2025/1734 of the European Central Bank of 31\xa0July 2025 on safeguards in relation to access by central counterparties to Eurosystem overnight credit in TARGET (ECB/2025/29)'},
 'summary': '',
 'summary_detail': {'type': 'text/html',
  'language': 'en',
  'base': 'https://eur-lex.europa.eu/EN/display-feed.rss?myRssId=zqe48ppy80IwdPmk3HJVPsD4fM281XNaoMTLQ6ifL58%3D',
  'value': ''},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://eur-lex.europa.eu/./legal-content/AUTO/?uri=CELEX:32025D1734'}],
 'link': 'https://eur-lex.europa.eu/

In [ ]:
new_feed_folder = os.path.join(os.getenv("RAW_DATA_PATH"), "eu/eurlex-feed", datetime.now().strftime("%Y%m%d"))
os.makedirs(new_feed_folder, exist_ok=True)
for entry in feed.entries:
    url = entry.link
    response = requests.get(url)
    celex = entry.title.split(":")[1].strip()
    filename = os.path.join(new_feed_folder, celex + ".html")  # sanitize filename

    with open(filename, "w", encoding="utf-8") as f:
        f.write(response.text)

meta_filename = os.path.join(new_feed_folder, "metadata.json")
with open(meta_filename, "w", encoding="utf-8") as f:
    json.dump(feed.entries, f, indent=4)

In [4]:
df = json_normalize(
    feed.entries,
    sep='_',  # use underscore to flatten nested dicts
    record_path=None,
    meta=None,
    errors='ignore'
)
df['published_dt'] = pd.to_datetime(df['published'], format = '%a, %d %b %Y %H:%M:%S %z', utc=True)
df = df.sort_values(by='published_dt', ascending=False).reset_index(drop=True)
# df = df[df['published_dt'] > datetime(2025, 1, 1, tzinfo=timezone.utc)]

In [5]:
tp_ls = [r['links'] for r in feed.entries]
tp_ls[0]

[{'rel': 'alternate',
  'type': 'text/html',
  'href': 'https://eur-lex.europa.eu/./legal-content/AUTO/?uri=CELEX:32025D1734'}]

In [18]:
import psycopg2
import os

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    dbname=os.getenv("DB_NAME")
)

OperationalError: connection to server at "reg-guru.c3my688ou3oy.ap-southeast-1.rds.amazonaws.com" (47.130.90.234), port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
cur = conn.cursor()
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO legal_acts (celex, title, link, published_dt, author, summary)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['celex'], row['title'], row['link_href'], row['published_dt'], row['author_name'], row['summary']))

conn.commit()
cur.close()
conn.close()